## Resample data into Healpix

1. Create HEALPix grid 
2. Extract data if it is still in a zip file
3. Interpolate from initial points to Healpix grid. Uses linear interpolation
4. Save file

In [2]:
import sys
sys.path.append('/'.join(sys.path[0].split('/')[:-1]))

import zipfile

import xarray as xr
import numpy as np
import os
from pathlib import Path
import healpy as hp

In [3]:
input_dir = '../data/equiangular/5.625deg/'
output_dir = '../data/healpix/5.625deg/'
nside = 16
n_pixels = 12*(nside**2)

In [4]:
# New HEALPix grid
n_pixels = 12*(nside**2)
hp_lon, hp_lat = hp.pix2ang(nside, np.arange(n_pixels), lonlat=True, nest=True)
lon_idx = xr.DataArray(hp_lon, dims=["lon"])
lat_idx = xr.DataArray(hp_lat, dims=["lat"])

In [5]:
all_files = os.listdir(input_dir)
all_files

['toa_incident_solar_radiation',
 'total_precipitation',
 'potential_vorticity',
 'temperature_850',
 'vorticity',
 'geopotential_500',
 '10m_v_component_of_wind',
 'constants',
 'u_component_of_wind',
 'total_cloud_cover',
 'specific_humidity',
 '2m_temperature',
 '10m_u_component_of_wind',
 'temperature',
 'v_component_of_wind',
 'all_5.625deg.zip',
 'relative_humidity',
 'geopotential']

In [6]:
#all_files = os.listdir(input_dir)
all_files = ['toa_incident_solar_radiation', 'temperature_850', 'geopotential_500', 'constants', 'total_cloud_cover']

#all_files = ['toa_incident_solar_radiation']
for f in all_files:
    print('Working on ', f)
    
    # Check if file has been unzip
    if not os.path.isfile(input_dir + f + '/' + f + '_1979_5.625deg.nc') and not os.path.isfile(input_dir + f + '/' + f + '_5.625deg.nc'):
        print('Data Extraction...')
        # extract data in the same folder 
        with zipfile.ZipFile(input_dir + f + '/' + f + '_5.625deg.zip',"r") as zip_ref:
            zip_ref.extractall(input_dir + f + '/')
            
    # Interpolate
    print('Interpolation...')
    ds = xr.open_mfdataset(input_dir + f + '/*.nc', combine='by_coords')
    
    #linear interpolation
    interp_ds = ds.interp(lon=('node', lon_idx), lat=('node', lat_idx)).interpolate_na(dim='node') 
    interp_ds = interp_ds.assign_coords(node=np.arange(n_pixels))
    
    # Create out folder
    out_path =  output_dir + f + "/"
    Path(out_path).mkdir(parents=True, exist_ok=True)
    
    # Save
    out_filename = f + '_5.625deg.nc'
    interp_ds.to_netcdf(out_path + out_filename)

Working on  toa_incident_solar_radiation
Interpolation...
Working on  temperature_850
Data Extraction...
Interpolation...
Working on  geopotential_500
Data Extraction...
Interpolation...
Working on  constants
Interpolation...
Working on  total_cloud_cover
Interpolation...
